# 引言

本章关注于Python在函数签名中的类型注解(Type Hints)。在Python中引入了显式的类型注解，可以为函数参数、返回值、变量等添加类型提示。主要目的在于帮助开发工具通过静态检查发现代码中的Bug。

# 新内容简介

全新内容。

由于静态类型系统的限制，PEP 484 引入了渐进类型系统(gradual type system)。

# 什么是渐进类型

PEP 484 引入了渐进类型系统。一个渐进类型系统：
* 是可选的 默认情况下，类型检查器不应该为没有类型注解的代码发出告警。当类型检查器不能决定对象的类型时，会假设它为`Any`类型。`Any`类型被认为与所有其他类型兼容。
* 不在运行时捕获类型错误 类型注解由静态类型检查器使用，IDE会发出告警。但它们不会防止不一致的值传递到函数或在运行时赋值给变量。
* 不增强性能 类型注解提供的数据理论上允许在生成的字节码上优化，但这种优化未在任何Python运行时中实现。



渐进类型最有用的特征在于注解总是可选的。

在静态类型系统中，大多数类型约束很容易表达，但还有一些是笨重的、一些是难的，还有一小部分是不可能的。你很可能会编写一段优秀的Python代码，具有良好的测试覆盖率和测试通过率，但仍无法增加使类型检查器满意的类型注解。

类型注解在所有级别上都是可选的：你的整个包可以没有类型注解，当你引入该包到使用类型注解的模块时可以静默类型检查器，也可以增加特殊的评论来类型检查器忽略代码中特定的行。

# 渐进类型实战

由一个简单的函数开始，然后逐渐增加类型注解。
`messages.py`
```py
def show_count(count, word):
    if count == 1:
        return f'1 {word}'
    count_str = str(count) if count else 'no'
    return f'{count_str} {word}s'
```


In [3]:
def show_count(count, word):
    if count == 1:
        return f'1 {word}'
    count_str = str(count) if count else 'no'
    return f'{count_str} {word}s'


show_count(99, 'bird')

'99 birds'

In [4]:
show_count(1, 'bird')

'1 bird'

In [5]:
show_count(0, 'bird')

'no birds'

## 由Mypy开始

在`messages.py`上运行`mypy`命令来开始类型检查：

In [10]:
! pip install mypy

In [9]:
! mypy ./messages/no_hints/messages.py

Success: no issues found in 1 source file


此时Mypy没有发现任何问题。

如果一个函数签名没有任何注解，Mypy默认会忽略它。下面，增加`pytest`单元测试，该代码在`messages_test.py`中：
```py
from pytest import mark

from messages import show_count

@mark.parametrize('qty, expected', [
    (1, '1 part'),
    (2, '2 parts'),
])
def test_show_count(qty, expected):
    got = show_count(qty, 'part')
    assert got == expected

def test_show_count_zero():
    got = show_count(0, 'part')
    assert got == 'no parts'
```

下面开始增加类型注解。

## 让Mypy更严格

命令行可选项`--disallow-untyped-defs`使Mypy标记任何对其所有参数和返回值没有类型注解的函数定义。

使用`--disallow-untyped-defs`在测试文件上产生三个error和一个note：

In [13]:
! mypy  --disallow-untyped-defs ./messages/no_hints/messages_test.py

messages\no_hints\messages.py:1: error: Function is missing a type annotation
messages\no_hints\messages_test.py:9: error: Function is missing a type annotation
messages\no_hints\messages_test.py:13: error: Function is missing a return type annotation
messages\no_hints\messages_test.py:13: note: Use "-> None" if function does not return a value
Found 3 errors in 2 files (checked 1 source file)


对于渐进类型的第一步，我倾向于使用另一个可选项：`--disallow-incomplete-defs`。初始时，它不会抱怨任何问题：

In [14]:
! mypy  --disallow-incomplete-defs ./messages/no_hints/messages_test.py

Success: no issues found in 1 source file


接着，我可以仅为`message.py`中的`show_count`增加返回类型：
```py
def show_count(count, word) -> str:
```
这足以让Mypy查看它了。使用前面同样的命令行去检查`messages_test.py`会让Mypy再次查看`messages.py`：

In [15]:
! mypy  --disallow-incomplete-defs ./messages/no_hints/messages_test.py

messages\no_hints\messages.py:1: error: Function is missing a type annotation for one or more arguments
Found 1 error in 1 file (checked 1 source file)


现在我们可以逐渐为函数增加类型注解，而不会得到我们没有标注的告警。下面是能满足Mypy的完整带注解签名：
```py
def show_count(count: int, word: str) -> str:
```

In [16]:
! mypy  --disallow-incomplete-defs ./messages/no_hints/messages_test.py

Success: no issues found in 1 source file


除了在命令行中使用可选项之外，还可以把你想要的可选项写到Mypy配置文件中。可以有全局设定和模块设定。
下面是一个简单的`mypy.ini`例子：
```
[mypy]
python_version = 3.9
warn_unused_configs = True
disallow_incomplete_defs = True
```

## 默认的参数值

示例中的`show_count`函数只用于(英文)常规名词。如果复数不能通过附加's'来拼写，我们应该让用户提供复数形式，像这样：
```py
>>> show_count(3, 'mouse', 'mice')
'3 mice'
```

让我们做一点"类型驱动开发"。首先，我们添加一个使用第三个参数的测试。不要忘记向测试函数中添加返回类型提示，否则Mypy将不会检查它。
```py
def test_irregular() -> None:
    got = show_count(2, 'child', 'children')
    assert got == '2 children'
```



In [18]:
! mypy  ./messages/hints_2/messages_test.py

messages\hints_2\messages_test.py:21: error: Too many arguments for "show_count"
Found 1 error in 1 file (checked 1 source file)


现在我们编辑`show_count`，增加可选的`plural`参数：
```py
def show_count(count: int, singular: str, plural: str = '') -> str:
    if count == 1:
        return f'1 {singular}'
    count_str = str(count) if count else 'no'
    if not plural:
        plural = singular + 's'
    return f'{count_str} {plural}'
```



In [19]:
! mypy  ./messages/hints_2/messages_test.py

Success: no issues found in 1 source file


现在Mypy就不会报错了。

## 使用None作为默认值

在上面，参数`plural`被注解为`str`，它的默认值为`''`，所以没有类型冲突。

这种方法不错，但在其他情况下，`None`是更好的默认值。如果可选的参数需要一个可变类型，那么`None`是唯一合理的默认值。

为了让`None`作为`plural`参数的默认值，新的签名变成：
```py
from typing import Optional

def show_count(count: int, singular: str, plural: Optional[str] = None) -> str:
```

我们拆开来看：
* `Optional[str]` 意味着`plural`可以为`str`或`None`
* 你必须显示地提供默认值` = None`

如果没有为`plural`赋予默认值，Python运行时会将它看成必传参数。记住：在运行时，忽略类型注解。

注意我们需要从`typing`引入`Optional`。在导入类型时，最好使用`from typing import X`的语法来减少函数签名的长度。

> `Optional`并不是一个很好的名称，因为它不会使参数成为可选的。使参数成为可选的原因是为该参数分配一个默认值。`Optional[str]`仅表示：此参数的类型可以是`str`或`None`T类型。

# 类型被定义来支持操作

实际上，将受支持的操作集合看成一种类型的定义特征更有用。

比如，从可用的操作角度来看，在下面的函数中，`x`的有效类型是什么？
```py
def double(x):
    return x * 2
```

`x`参数类型可能是数值(`int`、`complex`、`Fraction`和`numpy.uint32`等)，但也可能是一个序列(`str`、`tuple`、`list`和`array`)，一个N维的`numpy.array`，或其他实现或继承了`__mul__`方法能接受`int`参数的类型。

然而，考虑下面注解的`double`方法。先忽略返回类型，我们关注于参数类型：
```py
from collections import abc
def double(x: abc.Sequence):
    return x * 2
```

类型检查器会拒绝该代码。如果你告诉Mypy `x`是类型`abc.Sequence`，它会标记`x * 2`为错误因为`abc.Sequence`没有实现或继承`__mul__`方法。在运行时，该代码可以在具体的序列，比如`str`、`tuple`、`list`、`array`等，以及数值上正常运行，因为运行时忽略类型注解。但类型检查器只关心显示声明的，而`abc.Sequence`没有`__mul__`。

这就是本小节的标题是“类型被定义来支持操作”的原因。Python运行时接收任意类型作为`x`参数。`x * 2`可能正常执行，或抛出`TypeError`(如果`x`不支持该操作)。相比之下，Mypy会认为注解版本下`doulbe`中的`x * 2`是错误的，因为`x`的类型`abc.Sequence`不支持该操作。

在渐进类型系统中，我们有两种不同的观点相互作用：
* 鸭子类型(Duck typing) 这种观点被Smalltalk——首创的面向对象语言——以及Pyhton,JS,Ruby采纳。对象必须指定类型，但变量(包括参数)是无类型的。实际上，对象定义的类型不重要，重要的是它实际支持的操作。如果我们能调用`birdie.quack()`，那么`birdie`在该上下文中是鸭子。根据定义，只有在运行时尝试对对象进行操作时才强制鸭子类型。这比名义类型更灵活，但代价是在运行时允许更多的错误。
* 名义类型(Nominal typing) 该观点被C++、Java和C#采纳，也被注解的Python支持。对象和变量都有类型。但对象只在运行时存在，然后类型检查器只关心由类型注解标注的变量(和参数)的源码。如果`Duck`是`Bird`的子类，你可以分配一个`Duck`实例到标记为`Bird`的参数`birdie`。但在函数体中，
